## Fixations numerations fix

Change the numeration of trials once they are separated in Like and Dislike conditions (avoid breaks in trial number)

In [2]:
import numpy as np
import pandas as pd
from pandas.core.frame import DataFrame as DF
import seaborn as sns
sns.set(color_codes=True)
#np.random.seed(sum(map(ord, "distributions")))
from sklearn import linear_model  # packages for the logistic regression function to plot the logistic regression 
from sklearn.linear_model import LogisticRegression # packages for the logistic regression function to plot the logistic regression 
import scipy
from scipy import stats, integrate
from scipy.stats import mode
from scipy.stats.stats import pearsonr # Pearson's correlation
from copy import copy as copy
import operator as operator
import pylab

In [35]:
#Input has the fixation and expdat with breaks in trial numerations (due to separation in Like and Dislike conditions)
#data_fix = pd.read_csv('/Users/pradyumna/Documents/gitDocs/FF2018/Output/fixations_collapsed_FFA_Dislike_v1.csv') 
#data_expData = pd.read_csv('/Users/pradyumna/Documents/gitDocs/FF2018/Output/expdata_bin_FFA_Dislike_v1.csv') 

#for binned data
data_fix = pd.read_csv('/Users/bdmlab/Documents/GiTs/FF2018/InputData/fixations_collapsed_FFA_Dislike_v2.csv') 
data_expData = pd.read_csv('/Users/bdmlab/Documents/GiTs/FF2018/output/expdata_bin_FFA_Dislike_v2.csv') 

#for not binned data
# data_fix = pd.read_csv('/Users/bdmlab/Documents/GiTs/FF2018/InputData/fixations_collapsed_FFA_Like_v2.csv') 
# data_expData = pd.read_csv('/Users/bdmlab/Documents/GiTs/FF2018/InputData/expdata_FFA_Like_v2.csv') 

data_expData = data_expData.apply(pd.to_numeric, errors='ignore')
data_fix = data_fix.apply(pd.to_numeric, errors='ignore')

## Replace numerations

In [36]:
partNums = pd.unique(data_expData["parcode"])

new_fix_dataframe = data_fix.iloc[0:0]
new_expData_dataframe = data_expData.iloc[0:0]

for i in partNums:
    old_index_trial = []
    new_index_trial = []
    participant_trials = []
    
    participant_trials = data_expData.loc[data_expData['parcode'] == i]
    part_fix_trials = data_fix.loc[data_fix['parcode'] == i]
    
    old_index_trial = participant_trials['trial'].values
    new_index_trial = range(len(old_index_trial))
    
    for j in new_index_trial:
            #insert new column with new trial numeration : case trialData
            trialCol2Replace = participant_trials['trial'].replace(old_index_trial[j], new_index_trial[j])
            participant_trials = participant_trials.drop(['trial'], axis=1)
            participant_trials.insert(1,'trial',trialCol2Replace)
            
            #insert new column with new trial numeration : case Fixations
            fixCol2Replace = part_fix_trials['trial'].replace(old_index_trial[j], new_index_trial[j]).values
            part_fix_trials = part_fix_trials.drop(['trial'], axis=1)
            part_fix_trials.insert(1,'trial',fixCol2Replace)
            
            #print (participant_trials)
            time_trials = part_fix_trials['fix_time'].values
            
            ix = np.in1d(time_trials, 0)
            delete_fix_idx = np.where(ix)
            part_fix_trials = part_fix_trials.drop(part_fix_trials.index[delete_fix_idx])
            
    new_fix_dataframe = pd.concat([new_fix_dataframe, part_fix_trials],axis = 0)
    new_expData_dataframe = pd.concat([new_expData_dataframe, participant_trials], axis = 0)

In [ ]:
# # Output the correctly numerated trials for both CSV files
# new_expData_dataframe.to_csv("InputData/expdata_collapsed_FFA_Dislike_v2.csv",index = False)
# new_fix_dataframe.to_csv("InputData/fixations_collapsed_FFA_Dislike_v2.csv",index = False)


## Format appropiate for GLAM toolbox for fixations
Agregate fixations for each IA (similar to dwelling time) 

In [37]:
new_fix_dataframe

,parcode,trial,fix_item,fix_time
0,0,0,0,601
1,0,0,1,878
2,0,0,2,694
3,0,0,1,1154
4,0,1,3,215
5,0,1,1,686
6,0,1,2,1488
7,0,1,1,632
8,0,1,2,403
9,0,2,3,339


In [38]:
partNums = pd.unique(data_expData["parcode"])
partNums

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32])

In [39]:
IAFixMatrix = []

for i in partNums:
    PartIAFixMatrix= new_fix_dataframe.loc[(new_fix_dataframe['parcode'] == i )]
    trialNums = pd.unique(PartIAFixMatrix["trial"])
    for j in trialNums:
        PartTrialIAFixMatrix= PartIAFixMatrix.loc[(PartIAFixMatrix['trial'] == j )]
        # considering that IA 1 is left and IA 2 is right ( IAs 3 and 0 are not fixated in the item)
        # Sum left item
        LGazeTime = PartTrialIAFixMatrix.loc[PartTrialIAFixMatrix['fix_item'] == 1, 'fix_time'].sum() 
        # Sum right item
        RGazeTime = PartTrialIAFixMatrix.loc[PartTrialIAFixMatrix['fix_item'] == 2, 'fix_time'].sum()  
        LGaze = LGazeTime/(LGazeTime + RGazeTime)
        RGaze = RGazeTime/(LGazeTime + RGazeTime)
        
        IAFixMatrix.append([LGaze, RGaze]) 

In [40]:
IAFixDataFrame = pd.DataFrame(IAFixMatrix,columns=['gaze_0','gaze_1'])

In [41]:
GLAM_output = pd.concat([new_expData_dataframe, IAFixDataFrame], axis = 1)
GLAM_output = GLAM_output.replace(-1, 0)
GLAM_output = GLAM_output.rename(index=str, columns={"parcode": "subject", "item_left": "item_value_0", "item_right": "item_value_1"})

In [42]:
GLAM_output.to_csv("InputData/GlamDataFF2018_Dislike_Bin_v2.csv",index = True)